In [28]:
import torch
import torch.nn as nn
from torch.nn import functional as F

In [29]:
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt -O ~/input.txt

--2024-10-04 08:57:34--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘/Users/josemiguelvilchesfierro/input.txt’

/Users/josemiguelvi 100%[===================>]   1.06M  1.74MB/s    in 0.6s    

2024-10-04 08:57:35 (1.74 MB/s) - ‘/Users/josemiguelvilchesfierro/input.txt’ saved [1115394/1115394]



In [30]:
# read it in to inspect it
with open('/Users/josemiguelvilchesfierro/input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [31]:
#Hyperparameters
batch_size = 32 # number of sequences in a mini-batch
block_size = 8  # number of characters in a sequence
max_iter = 3000 # number of training iterations
eval_interval = 300
learning_rate = 1e-2
eval_iters = 200
n_emb = 32

In [32]:
list_ch = sorted(list(set(text)))
vocab_size = len(list_ch)
stoi = {ch:i for i,ch in enumerate(list_ch)}
itos = {i:ch for i,ch in enumerate(list_ch)}
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda h: ''.join([itos[i] for i in h])

In [33]:
#Encode dataset
text_encode = torch.tensor(encode(text), dtype=torch.long)
#Create a dataset
n = int(len(text_encode) * 0.9)
train = text_encode[:n]
valid = text_encode[n:]

In [34]:
torch.manual_seed(1337)
def get_batch(split):
    data = train if split == 'train' else valid
    ix = torch.randint(len(data)- block_size, (batch_size,))
    X = torch.stack([data[   i : i+block_size   ] for i in ix])
    Y = torch.stack([data[i + 1: i+block_size +1] for i in ix])
    return X, Y

In [35]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [36]:
class BigramLanguajeModel(nn.Module):

    def __init__(self):
        super().__init__()
        self.embedding_tokken = nn.Embedding(vocab_size, n_emb)
        self.lm_head = nn.Linear(n_emb, vocab_size) 

    def forward(self, idx, targets= None):        
        #Embedding tokens and targets in a 3d tensor
        emb = self.embedding_tokken(idx) # B T C [4x 8 x 32]
        #Linear layer
        logits = self.lm_head(emb) # B T C [4 x 8 x 65]

        #Reshape the tensor to 2d for cross entropy
        if targets is None:
            loss = None 
        else:
            B , T, C = logits.shape
            logits = logits.view(B*T , C) #[32 x 65]
            targets = targets.view(B*T) #[32]
            #Calculate loss
            loss = F.cross_entropy(logits, targets)
        
        return logits, loss

    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
            #get the prediction
            logits, loss = self(idx) # [B x T x C] because targets is None [4 x 8 x 65]
            #get the last token
            logits = logits[:, -1, :] # Becomes [B x C] [4 x 65]
            #softmax
            probs = F.softmax(logits, dim=1) 
            #sample
            new_token = torch.multinomial(probs, 1)
            #append to the sequence
            idx = torch.cat([idx, new_token], dim=1) # B x T+1   

        return idx

In [37]:
model = BigramLanguajeModel()

In [38]:
optimier = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [39]:
for i in range(max_iter):

    # every once in a while evaluate the loss on train and val sets
    if (i % eval_interval == 0) or (i == max_iter - 1):
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    #forward step
    xb, yb = get_batch('train')
    logits, loss = model.forward(xb, yb)


    # Print the loss every 1000 epochs
    #if i % 1000 == 0:
    #    print(f'Epoch [{i}/{max_iter}], Loss: {loss.item():.4f}')
    
    #backward step
    optimier.zero_grad(set_to_none=True)
    loss.backward()
    optimier.step()


step <built-in function iter>: train loss 4.3886, val loss 4.3734
step <built-in function iter>: train loss 2.5268, val loss 2.5406
step <built-in function iter>: train loss 2.5004, val loss 2.5331
step <built-in function iter>: train loss 2.4910, val loss 2.5101
step <built-in function iter>: train loss 2.4985, val loss 2.5155
step <built-in function iter>: train loss 2.4819, val loss 2.5046
step <built-in function iter>: train loss 2.4864, val loss 2.5177
step <built-in function iter>: train loss 2.4870, val loss 2.5024
step <built-in function iter>: train loss 2.4892, val loss 2.5159


KeyboardInterrupt: 

In [ ]:
idx = torch.zeros((1,1), dtype=torch.long)
generate = model.generate(idx, 400)[0].tolist()
print(decode(generate))


Hade d mar'd, wulisppe thoto lower, CEER:
ME cirmu s tose sind II ounobe Mathe ond atheroul lle? be tcous,
Dscr

NVOr fis,itheman'sthil F st ge Dusty.
To grs m; borory othathonthe dech ges, t byoullaious. HAndar wareral momy Thiseryuse MNThe out barie therk
Pas, wit a and n
Thys thod it, mald dowokivithas, ines ld beromanth t s t;
Yrrayorpirrorory cetet.
ADuliond it'd n,
RLBEThovedr:

SAnt acheawa


In [14]:
B, T, C = 4, 8, 2
X = torch.randn((B, T, C))
X

tensor([[[ 1.0260, -0.3394],
         [-0.3419,  1.4727],
         [ 1.2524, -0.1829],
         [-2.2855, -0.1792],
         [-0.7449,  0.2584],
         [ 0.1915,  0.6320],
         [-0.2829,  0.5509],
         [ 0.1462,  1.2191]],

        [[-1.4528,  0.6634],
         [ 0.3494, -0.1100],
         [-1.8360, -0.5359],
         [ 1.0145, -0.6041],
         [-1.0142,  2.0075],
         [-0.1028, -0.5041],
         [-0.3768, -0.8460],
         [ 1.0721, -1.7458]],

        [[ 0.6611, -0.7424],
         [ 0.5601,  0.1108],
         [ 0.8876, -0.9288],
         [-2.1880,  0.6263],
         [ 1.2003, -0.6963],
         [-0.3036,  0.2314],
         [ 1.2972,  0.8315],
         [ 1.8056,  2.1135]],

        [[-2.7030, -2.6175],
         [-0.1273,  1.4003],
         [-0.6449,  0.2759],
         [ 3.0531,  0.2194],
         [ 1.4805, -0.8108],
         [ 1.6489,  1.0751],
         [ 1.2669,  0.3340],
         [-1.4834,  0.6269]]])

In [15]:
xbow = torch.zeros(B,T,C)
for b in range(B):
    for t in range(T):
        xprev = X[b,: t+1]    
        xbow[b,t] = torch.mean(xprev, dim=0)
        print(f"b = {b}")
        print(f"t = {t+1}")
        print(xprev)
        print(f"{torch.mean(xprev, dim=0)}")
        print(f"xprev = {xprev.shape}")
        print("------------------")

b = 0
t = 1
tensor([[ 1.0260, -0.3394]])
tensor([ 1.0260, -0.3394])
xprev = torch.Size([1, 2])
------------------
b = 0
t = 2
tensor([[ 1.0260, -0.3394],
        [-0.3419,  1.4727]])
tensor([0.3421, 0.5666])
xprev = torch.Size([2, 2])
------------------
b = 0
t = 3
tensor([[ 1.0260, -0.3394],
        [-0.3419,  1.4727],
        [ 1.2524, -0.1829]])
tensor([0.6455, 0.3168])
xprev = torch.Size([3, 2])
------------------
b = 0
t = 4
tensor([[ 1.0260, -0.3394],
        [-0.3419,  1.4727],
        [ 1.2524, -0.1829],
        [-2.2855, -0.1792]])
tensor([-0.0872,  0.1928])
xprev = torch.Size([4, 2])
------------------
b = 0
t = 5
tensor([[ 1.0260, -0.3394],
        [-0.3419,  1.4727],
        [ 1.2524, -0.1829],
        [-2.2855, -0.1792],
        [-0.7449,  0.2584]])
tensor([-0.2188,  0.2059])
xprev = torch.Size([5, 2])
------------------
b = 0
t = 6
tensor([[ 1.0260, -0.3394],
        [-0.3419,  1.4727],
        [ 1.2524, -0.1829],
        [-2.2855, -0.1792],
        [-0.7449,  0.2584],
  

In [16]:
torch.manual_seed(42)

tril = torch.tril(torch.ones(T,T))
wei = torch.zeros((T,T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=1)
wei

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3333, 0.3333, 0.3333, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2500, 0.2500, 0.2500, 0.2500, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2000, 0.2000, 0.2000, 0.2000, 0.2000, 0.0000, 0.0000, 0.0000],
        [0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.0000, 0.0000],
        [0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.0000],
        [0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250]])